## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Bing Grounding Example

This notebook demonstrates how to create an Azure AI agent that uses Bing Grounding search to find real-time information from the web.

## Features Covered:
- Setting up Bing Grounding search integration
- Using HostedWebSearchTool for web searches
- Creating agents that can access current web information
- Proper resource management with async context managers

## Prerequisites

Before running this notebook, ensure you have:
- Azure CLI installed and authenticated (`az login --use-device-code`)
- Access to an Azure AI Foundry project with deployed models
- Environment variables set up (see Initial Setup section below)

This example demonstrates Azure AI agents with Bing Grounding for web search capabilities. The agent can search the web for current information and provide informed responses.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import os
from pathlib import Path
from agent_framework import ChatAgent, HostedWebSearchTool
from agent_framework_azure_ai import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [2]:
from pathlib import Path  # For working with file paths
import os  # For environment variables
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

# Get configuration from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment = os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o")

# Validate and clean endpoint (remove any inline comments)
if project_endpoint and '#' in project_endpoint:
    print("⚠️  WARNING: Endpoint contains a '#' character - cleaning it...")
    project_endpoint = project_endpoint.split('#')[0].strip()

print(f"🔗 Project Endpoint: {project_endpoint}")
print(f"🤖 Model Deployment: {model_deployment}")

🔗 Project Endpoint: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject
🤖 Model Deployment: gpt-4o


## Check Environment Setup

Let's verify that the required environment variable is set:

In [ ]:
import os

# Check if BING_CONNECTION_ID is set
bing_connection_id = os.getenv('BING_CONNECTION_ID')
if bing_connection_id:
    print("✅ BING_CONNECTION_ID is set")
    print(f"Connection ID: {bing_connection_id[:50]}...")  # Show first 50 chars for security
else:
    print("❌ BING_CONNECTION_ID environment variable is not set")
    print("Please set this variable before proceeding.")

❌ BING_CONNECTION_ID environment variable is not set
Please set this variable before proceeding.


: 

## Create Bing Grounding Search Tool

The `HostedWebSearchTool` allows our agent to perform web searches using Bing. The connection ID will be automatically picked up from the environment variable:

In [ ]:
# Create Bing Grounding search tool using HostedWebSearchTool
# The connection_id will be automatically picked up from BING_CONNECTION_ID environment variable
bing_search_tool = HostedWebSearchTool(
    name="Bing Grounding Search",
    description="Search the web for current information using Bing",
)

print("✅ Bing search tool created successfully")

## Create and Run the Agent

Now let's create an Azure AI agent that can use the Bing search tool to find current information from the web:

In [ ]:
async def main() -> None:
    """Main function demonstrating Azure AI agent with Bing Grounding search."""
    
    # Use AzureAIAgentClient as async context manager for automatic cleanup
    async with (
        AzureAIAgentClient(async_credential=AzureCliCredential()) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        # Demonstrate agent capabilities with web search
        print("=== Azure AI Agent with Bing Grounding Search ===\n")

        user_input = "What is the most popular programming language?"
        print(f"User: {user_input}")
        response = await agent.run(user_input)
        print(f"Agent: {response.text}\n")

## Execute the Example

Run the main function to see the agent in action:

In [ ]:
# Run the main function
await main()

## Try Additional Queries

Let's test the agent with more queries that require current information:

In [ ]:
async def test_more_queries():
    """Test the agent with additional queries that require web search."""
    
    async with (
        AzureAIAgentClient(async_credential=AzureCliCredential()) as client,
        ChatAgent(
            chat_client=client,
            name="BingSearchAgent",
            instructions=(
                "You are a helpful assistant that can search the web for current information. "
                "Use the Bing search tool to find up-to-date information and provide accurate, "
                "well-sourced answers. Always cite your sources when possible."
            ),
            tools=bing_search_tool,
        ) as agent,
    ):
        queries = [
            "What are the latest trends in artificial intelligence?",
            "What is the current weather in Seattle?",
            "What are the latest Microsoft announcements?"
        ]
        
        for query in queries:
            print(f"\n--- Query: {query} ---")
            response = await agent.run(query)
            print(f"Agent: {response.text}")
            print("-" * 50)

# Run additional queries
await test_more_queries()

## Key Takeaways

1. **Bing Grounding Integration**: The `HostedWebSearchTool` provides seamless integration with Bing search capabilities
2. **Environment Configuration**: The `BING_CONNECTION_ID` environment variable is crucial for connecting to your Azure AI project's Bing resource
3. **Real-time Information**: Agents can now access current web information, making them more useful for time-sensitive queries
4. **Source Citation**: The agent is instructed to cite sources, promoting transparency and reliability
5. **Resource Management**: Using async context managers ensures proper cleanup of resources
6. **Tool Integration**: The Bing search tool seamlessly integrates with the agent's existing capabilities

## Troubleshooting

If you encounter issues:
1. **Authentication**: Ensure you're logged in with `az login`
2. **Environment Variable**: Double-check that `BING_CONNECTION_ID` is correctly set
3. **Azure AI Project**: Verify that Bing Grounding is properly configured in your Azure AI Foundry project
4. **Permissions**: Ensure you have the necessary permissions to access the connected Bing search resource